In [14]:
!pip install langchain-openai langchain_openai langchain_community langchain_qdrant qdrant_client

In [44]:
# Test that your OpenAI API key is correctly set as an environment variable
# Note. if you run this notebook locally, you will need to reload your terminal and the notebook for the env variables to be live.
import os

# Note. alternatively you can set a temporary env variable like this:
#os.environ["OPENAI_API_KEY"] = ""

os.environ["AZURE_OPENAI_API_KEY"]=""
os.environ["AZURE_OPENAI_ENDPOINT"]=""
os.environ["OPENAI_API_VERSION"]="2024-12-01-preview"
os.environ["OPENAI_MODEL"]="gpt-4.1-mini"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]="gpt-4.1-mini"
#os.environ["OPENAI_MODEL"]="text-embedding-3-small"

if os.getenv("AZURE_OPENAI_API_KEY") is not None:
    print("AZURE_OPENAI_API_KEY is ready")
else:
    print("AZURE_OPENAI_API_KEY environment variable not found")

AZURE_OPENAI_API_KEY is ready


In [27]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance
from qdrant_client.models import VectorParams

openai_embeddings = AzureOpenAIEmbeddings(
    model='text-embedding-3-large',
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    openai_api_version=os.environ["OPENAI_API_VERSION"],
)
# initialize the qdrant client
client = QdrantClient(
    host="qdrant",
    prefer_grpc=True,
)

if not client.collection_exists(collection_name):
    client.create_collection(
       collection_name=collection_name,
       vectors_config=VectorParams(
           size=3072,
           distance=Distance.COSINE
        ),
    )
# create the vector store
vectordb = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=openai_embeddings,
)

In [41]:
from langchain.chains import create_history_aware_retriever
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI

collection_name = "k8s-official-doc"

llm = AzureChatOpenAI(
    temperature="0.0",
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    #rate_limiter=InMemoryRateLimiter(
    #    requests_per_second=60,
    #    check_every_n_seconds=1,
    #)
)

chat_history = []

condense_question_system_template = """
    給定一段對話歷史和最新的用戶問題，用戶問題可能參考對話歷史中的內容，
    重新構造一個獨立的問題，該問題可以在沒有對話歷史的情況下理解。
    不要直接回答問題，只有在需要時重新構造問題，否則原樣返回。
"""

condense_question_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", condense_question_system_template),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

system_prompt = """
    你是一個問答任務的助手。根據 Retrival 的資料內容回答問題。
    根據內容準確回答問題，確保答案清晰且準確，避免加入任何虛構或未經證實的資訊。
    維持一段落輸出，不要有換行符號。
    保留內容重要資訊，如文章 title，顯示原文 title 不需要翻譯。
    保留內容重要資訊，如url，請列出不重複的url。
    有程式碼的部分，請列出程式碼。
    如果內容中沒有直接回答，請根據相關資訊進行合理推斷，但不要編造答案。
    使用符合台灣用語習慣的表達方式，提高可讀性。
    確保最終輸出內容為台灣繁體中文。如果是英文專有名詞，可以在字詞後加註原文。
    例如：「人工智慧（Artificial Intelligence）」。

    {context}
"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

vb_retriever = vectordb.as_retriever(
    #search_type="similarity",
    search_type="mmr",
    search_kwargs={
        "k": 4,
        #"score_threshold": 0.5,
    }
)

# Create a history-aware retriever from the OpenAI model and the retriever
convo_qa_chain = create_retrieval_chain(
    create_history_aware_retriever(
        llm,
        vb_retriever,
        condense_question_prompt
    ),
    create_stuff_documents_chain(
        llm,
        qa_prompt
    )
)

In [42]:
def generate_prompt(question):
    if not chat_history:
        prompt = f"""
        問題：{question}
        """
    else:
        print("Chat with previous history")
        context_entries = [f"Question: {q}\nAnswer: {a}" for q, a in chat_history[-3:]]
        context = "\n\n".join(context_entries)
        prompt = f"""
            使用最近對話提供的內容，以簡潔且具信息性的方式回答新問題。
            最近對話的內容：{context}
            新問題：{question}
            答案：
        """

    return prompt

def ask_question(question):
    prompt = generate_prompt(question)
    # Invoke the chain with the question and the chat history
    result = convo_qa_chain.invoke(
        {
            "input": prompt,
            "chat_history": chat_history
        }
    )

    # Return the bot's answer
    return result["answer"]

In [43]:
ask_question("how to extend-kubectl?")

'根據Retrieval資料，沒有直接提到「how to extend-kubectl」的具體步驟或方法。不過，一般來說，擴充 kubectl（Kubernetes 命令列工具）通常可以透過以下方式進行：1. 使用 kubectl 插件（plugins），kubectl 支援自訂插件，您可以將可執行檔放在系統路徑中，並以 kubectl-<plugin-name> 命名，然後透過 kubectl <plugin-name> 執行。2. 利用 kubectl 的自訂資源（Custom Resources）和自訂控制器（Custom Controllers）來擴充 Kubernetes 的功能，間接擴展 kubectl 的操作範圍。3. 使用 kubectl 的外掛機制，例如 krew 插件管理器，方便安裝和管理各種 kubectl 插件。若需要更詳細的操作步驟，建議參考官方文件或相關技術文章。'

In [ ]:
glob_pattern="./**/*.md"):
directory_path = "website/content/en/docs/
glob_pattern = glob_pattern
documents = []
all_sections = []

from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader

loader = DirectoryLoader(
    self.directory_path,
    glob=self.glob_pattern,
    show_progress=True,
    recursive=True,
    loader_cls=UnstructuredMarkdownLoader,
    loader_kwargs={"mode":"single"},
    #loader_kwargs={"mode":"elements"},
)

documents = loader.load()